In [ ]:
import pandas as pd
df = pd.read_csv('sms.csv', encoding='ISO-8859-1')
df = df.iloc[:,0:2]
df.head()

,class,bodytext
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.isnull().sum()

class       0
bodytext    0
dtype: int64

In [ ]:
import string
def punc_per(str1):
  cnt = sum([1 for i in str1 if i in string.punctuation])
  return round((cnt/(len(str1)-str1.count(' '))*100),0)
punc_per('hello!!!!')

44.0

In [ ]:
df['punct'] = df['bodytext'].apply(lambda x: punc_per(x))
df['len'] = df['bodytext'].apply(lambda x: len(x)-x.count(' '))
df.head(12)

,class,bodytext,punct,len
0,ham,"Go until jurong point, crazy.. Available only ...",10.0,92
1,ham,Ok lar... Joking wif u oni...,25.0,24
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,5.0,128
3,ham,U dun say so early hor... U c already then say...,15.0,39
4,ham,"Nah I don't think he goes to usf, he lives aro...",4.0,49
5,spam,FreeMsg Hey there darling it's been 3 week's n...,7.0,117
6,ham,Even my brother is not like to speak with me. ...,3.0,62
7,ham,As per your request 'Melle Melle (Oru Minnamin...,4.0,135
8,spam,WINNER!! As a valued network customer you have...,5.0,133
9,spam,Had your mobile 11 months or more? U R entitle...,2.0,126


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [ ]:
def clean_text(txt):
  data = [x for x in txt if x not in string.punctuation]
  data = ''.join(data)
  data = [ps.stem(x) for x in data.split() if x not in sw]
  return data
clean_text('waht is this>?????')  

['waht']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(analyzer=clean_text)
vector_output = cv.fit_transform(df['class'])
vectr_output = pd.DataFrame(vector_output.toarray(), columns = cv.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#train-test-split
from sklearn.model_selection import train_test_split
X= df.iloc[:,1:4]
y= df['class']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=0)

# Instantiate and fit TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer = clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['bodytext'])

# Use fitted TfidfVectorizer to transform body text in X_train and X_test
tfidf_train = tfidf_vect.transform(X_train['bodytext'])
tfidf_test = tfidf_vect.transform(X_test['bodytext'])

# Recombine transformed body text with body_len and punct% features
X_train = pd.concat([X_train[['len', 'punct']].reset_index(drop = True), pd.DataFrame(tfidf_train.toarray())], axis = 1)
X_test = pd.concat([X_test[['len', 'punct']].reset_index(drop = True), pd.DataFrame(tfidf_test.toarray())], axis = 1)

In [ ]:
X_test[8:9]

,len,punct,0,1,2,3,4,5,6,7,...,6983,6984,6985,6986,6987,6988,6989,6990,6991,6992
8,33,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf_model = rf.fit(X_train,y_train)
y_pred=rf_model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
print('String: ',df['bodytext'][8])
print('Actual: ',df['class'][8])
print('Prediction: ',rf_model.predict(X_test[8:9])[0])

String:  WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Actual:  spam
Prediction:  ham


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
print('Accuracy: ',round(accuracy_score(y_test,y_pred)*100,2))
print('Confusoin matrix: \n',confusion_matrix(y_test,y_pred))

Accuracy:  97.06
Confusoin matrix: 
 [[1196    0]
 [  41  156]]
